In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os


In [2]:
os.listdir('D:/动漫二次元头像数据集/faces/faces')[:10]

['0000fdee4208b8b7e12074c920bc6166-0.jpg',
 '0001a0fca4e9d2193afea712421693be-0.jpg',
 '0001d9ed32d932d298e1ff9cc5b7a2ab-0.jpg',
 '0001d9ed32d932d298e1ff9cc5b7a2ab-1.jpg',
 '00028d3882ec183e0f55ff29827527d3-0.jpg',
 '00028d3882ec183e0f55ff29827527d3-1.jpg',
 '000333906d04217408bb0d501f298448-0.jpg',
 '0005027ac1dcc32835a37be806f226cb-0.jpg',
 '0008242b08dabe9b5ebf145eefd74d3b-0.jpg',
 '000a7ac0c73b86812c0f94895ebe9e5a-0.jpg']

In [3]:
image_path = glob.glob('D:/动漫二次元头像数据集/faces/faces/*.jpg') #获取所有图片的路径

In [4]:
len(image_path)

51223

In [5]:
def load_preprosess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.cast(image,tf.float32)
    image = (image / 127.5) - 1
    return image

In [6]:
image_ds = tf.data.Dataset.from_tensor_slices(image_path)

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [8]:
image_ds = image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)

In [9]:
image_ds

<ParallelMapDataset shapes: (None, None, 3), types: tf.float32>

In [10]:
BATCH_SIZE = 64 
image_count = len(image_path)

In [11]:
image_ds = image_ds.shuffle(image_count).batch(BATCH_SIZE)
image_ds = image_ds.prefetch(AUTOTUNE)

In [12]:
def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(6*6*512, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((6, 6, 512)))
  

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())                 #6*6*256

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())                #12*12*128
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())                #24*24*64
    
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())                #48*48*32

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
                                                #96*96*3

    return model

In [13]:
generator = generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 18432)             1843200   
_________________________________________________________________
batch_normalization (BatchNo (None, 18432)             73728     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 18432)             0         
_________________________________________________________________
reshape (Reshape)            (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 6, 6, 256)         3276800   
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 256)         1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 6, 6, 256)         0

In [14]:
noise = tf.random.normal([1,100])
generated_image = generator(noise, training=False)
plt.imshow((generated_image[0, :, :, :3]+1)/2)

InternalError: Blas GEMM launch failed : a.shape=(1, 100), b.shape=(100, 18432), m=1, n=18432, k=100 [Op:MatMul]

In [ ]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same',
                                        input_shape=(96,96,3)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))     #48*48*32

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))    #
    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(516, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.GlobalAveragePooling2D())
    
    model.add(layers.Dense(1024))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = discriminator_model()
decision = discriminator(generated_image)   #对模型进行测试
discriminator.summary()

In [ ]:
print(decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_out,fake_out):
    
    #这里希望让真实图像被判定为1,假的图像判定为0
    real_loss = cross_entropy(tf.ones_like(real_out),real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out),fake_out)
    
    return real_loss + fake_loss

In [ ]:
#生成器接收假的的图片并希望能够被判为真
def generator_loss(fake_out):
    return cross_entropy(tf.ones_like(fake_out),fake_out)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)   #创建优化器

In [ ]:
EPOCHS = 800
NOISE_DIM = 100

num_exp_to_generator = 4  #生成观察样本

seed = tf.random.normal([num_exp_to_generator,NOISE_DIM]) #生成4个长度为100的随机向量

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE,NOISE_DIM])
    with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
         
        real_out = discriminator(images,training=True)
        gen_image = generator(noise,training=True)
        fake_out = discriminator(gen_image,training=True)
         
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out,fake_out)
        
    gradient_gen = gen_tape.gradient(gen_loss,generator.trainable_variables)         #gen_loss对generator.trainable_variables自动求微分其实就是dy比dx
    gradient_disc = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradient_gen,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradient_disc,discriminator.trainable_variables))
    

In [ ]:
def generate_and_save_images(model,epoch,test_input):
    predictions = model(test_input,training=False)
    
    fig = plt.figure(figsize=(6,6))
    
    for i in range(predictions.shape[0]):
        plt.subplot(2,2,i+1)
        plt.imshow((predictions[i, :, :,]+1)/2)
        plt.axis('off')
        plt.show()

In [ ]:
def train(dataset,epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
            
        if epoch%10 ==0:
            generate_and_save_images(generator,epoch +1,seed)
           
    generate_and_save_images(generator,epochs,seed)
            

In [ ]:
train(image_ds,EPOCHS)